In [3]:
import numpy as np
import pandas as pd

In [18]:
#读入数据 u.data存放了每一个用户对每一部电影的评级
header = ['user_id', 'item_id', 'rating', 'timestamp']
df=pd.read_csv('/Users/xwx/Desktop/learning/doing/推荐算法/ml-100k/u.data', sep='\t', names=header)

In [19]:
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [21]:
#看看多少不重复的用户和电影数量
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items)  )
    

Number of users = 943 | Number of movies = 1682


In [28]:
#根据测试样本的比例（test_size），本例中是0.25，来将数据混洗并分割成两个数据集
from sklearn.model_selection import train_test_split
train_data, test_data=train_test_split(df,test_size=0.25)
train_data.head()

,user_id,item_id,rating,timestamp
51915,407,214,4,875042466
52104,521,96,4,884477853
38621,256,1046,4,882164927
19573,399,273,3,882340657
85499,606,833,5,887060394


# 基于内存的协同过滤：根据历史数据找到相似

In [30]:
#建立用户-产品矩阵

#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():#将dataframe的每一行转换成tuple进行迭代
    train_data_matrix[line[1]-1, line[2]-1] = line[3]  #行列分别代表用户与产品，矩阵上写打分
    
test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

In [31]:
train_data_matrix

array([[5., 3., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [32]:
#创建相似性矩阵
from sklearn.metrics.pairwise import pairwise_distances
#当创建用户之间的相关性矩阵时，每一行应该代表一个用户对所有产品的打分，喜欢的产品越接近、用户越相似
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
#当创建产品之间的相关性矩阵时，每一行代表一个产品被所有用户的打分，产品的相似性通过观察所有对两个产品之间的打分的用户来度量
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [ ]:
#基于用户的CF过滤需要为用户的打分做一个调整，某些用户可能会倾向于对所有的电影，总是给予高或低评分。
#这些用户提供的评分的相对差比绝对评分值更重要。
#举个例子：假设，用户_k_对他最喜欢的电影打4星，而对所有其他的好电影打3星。
#现在假设另一个用户_t_对他/她喜欢的电影打5星，而对他/她感到无聊的电影打3星。
#那么这两个用户可能品味非常相似，但对打分系统区别对待
def predict(ratings, similarity, type='user'):
    if type == 'user':#如果是用户的CF
        mean_user_rating = ratings.mean(axis=1)#对所有用户在同一产品的打分求均值
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred